In [14]:
import Pkg; 

if split(pwd(),"/")[end] == "random_walk"
    cd(joinpath(@__DIR__, "../../../"))
    Pkg.activate("Project.toml")
end

using JLD2
using MorphoMolNotebooks
using MorphoMol
using CairoMakie
using LinearAlgebra
using Random
using Rotations
using StaticArrays
using Distributions
using Distances

In [15]:
function solvation_free_energy_with_interface_persistence_and_measures(x::Vector{Float64}, template_centers::Matrix{Float64}, radii::Vector{Float64}, rs::Float64, prefactors::AbstractVector, overlap_jump::Float64, overlap_slope::Float64, persistence_weights::Vector{Float64}, delaunay_eps::Float64)
    n_atoms_per_mol = size(template_centers)[2]
    flat_realization = MorphoMol.Utilities.get_flat_realization(x, template_centers)
    points = Vector{Vector{Float64}}([e for e in eachcol(reshape(flat_realization, (3, Int(length(flat_realization) / 3))))])
    idgm, ifil = MorphoMol.Energies.get_interface_diagram_and_filtration(points, n_atoms_per_mol)
    idgm = [idgm[2], idgm[3]]
    ifil = [(c.vertices, c.value) for c in ifil]
    measures = MorphoMol.Energies.get_geometric_measures_and_overlap_value(flat_realization, n_atoms_per_mol, radii, rs, overlap_jump, overlap_slope, delaunay_eps)
    sum(measures .* [prefactors; [1.0]]) + MorphoMol.Energies.get_total_persistence(idgm, persistence_weights), Dict("Vs" => measures[1], "As" => measures[2], "Cs" => measures[3], "Xs" => measures[4], "OLs" => measures[5], "IDGMs"  => idgm, "IFILs" => ifil)
end

solvation_free_energy_with_interface_persistence_and_measures (generic function with 1 method)

In [16]:
mol_type = "6r7m"
template_centers = MorphoMol.Utilities.TMV_TEMPLATES[mol_type]["template_centers"]
template_radii = MorphoMol.Utilities.TMV_TEMPLATES[mol_type]["template_radii"]
n_mol = 2
bounds = 80.0
x_init = MorphoMol.Utilities.get_initial_state(n_mol, bounds)
n_atoms_per_mol = length(template_radii)
radii = vcat([template_radii for i in 1:n_mol]...);

In [12]:
perturb_all(x, Σ) = x .+ (randn(length(x)) .* Σ)

function perturb_single_randomly_chosen(x, σ_r, σ_t)
    x_cand = deepcopy(x)
    i  = rand(0:(length(x)÷6)-1)
    x_cand[(i*6)+1:(i*6)+6] = x_cand[(i*6)+1:(i*6)+6] .+ (randn(6) .* [σ_r, σ_r, σ_r, σ_t, σ_t, σ_t])
    x_cand
end

perturb_single_randomly_chosen (generic function with 1 method)

In [13]:
simulation_time_minutes = 0.25

T = 2.5
β = 1.0 / T

rs = 1.4
η = 0.3665
pf = MorphoMol.Energies.get_prefactors(rs, η)
overlap_slope = 1.0
bounds = 150.0
persistence_weights = [0.1, 0.1]

σ_r = 0.15
σ_t = 1.25
Σ = vcat([[σ_r, σ_r, σ_r, σ_t, σ_t, σ_t] for _ in 1:n_mol]...)

energy(x) = solvation_free_energy_with_interface_persistence_and_measures(x, template_centers, radii, rs, pf, 0.0, overlap_slope, persistence_weights, 1.0)
perturbation(x) = perturb_single_randomly_chosen(x, σ_r, σ_t)

input = Dict(
    "template_centers" => template_centers,
    "template_radii" => template_radii,
    "n_mol" => n_mol,
    "σ_r" => σ_r,
    "σ_t" => σ_t,
    "rs" => rs,
    "η" => η,
    "white_bear_prefactpors" => pf,
    "overlap_slope" => overlap_slope,
    "persistence_weights" => persistence_weights,
    "T" => T,
    "mol_type" => mol_type
)

output = Dict{String, Vector}(
    "states" => Vector{Vector{Float64}}([]),
    "Es" => Vector{Float64}([]), 
    "Vs" => Vector{Float64}([]), 
    "As" => Vector{Float64}([]), 
    "Cs" => Vector{Float64}([]), 
    "Xs" => Vector{Float64}([]),
    "OLs" => Vector{Float64}([]),
    "αs" => Vector{Float32}([]),
    "IDGMs"  => Vector{Any}([]),
    "IFILs" => Vector{Any}([]),
)

rwm = MorphoMol.Algorithms.RandomWalkMetropolis(energy, perturbation, β)
simulation_time_minutes = 1.0

MorphoMol.Algorithms.simulate!(rwm, x_init, simulation_time_minutes, output);
@save "../../Data/tmp/wip_test.jld2" input output
output["αs"][end]

0.6418919f0

In [17]:
output["IFILs"]

96-element Vector{Any}:
 [([748, 1725], 23.92455349903287), ([748, 1724], 23.96354401541596), ([748, 1718], 24.28196954629767), ([846, 1724], 24.752825160736354), ([846, 1725], 24.800319910112158), ([740, 1725], 25.10700485325493), ([737, 1718], 25.213469938876298), ([869, 1724], 25.32574800184744), ([703, 1718], 25.45647706768041), ([740, 1718], 25.462829822196742)  …  ([798, 1464, 2004, 2017], 46.36206014270773), ([798, 1509, 2003, 2004], 46.4520948255289), ([810, 2038, 2067, 2098], 47.30434541547556), ([469, 789, 1566, 1995], 47.457641911347004), ([468, 1566, 1675, 1995], 47.62720496168401), ([810, 831, 2038, 2098], 48.301855104675305), ([468, 1205, 1674, 1995], 49.14033289963158), ([797, 798, 2003, 2004], 50.84849316210928), ([469, 789, 1995, 2003], 54.48503911056762), ([468, 469, 1566, 1995], 54.77862226711244)]
 [([51, 1718], 25.014294323285053), ([50, 1718], 25.098467849699105), ([748, 1725], 25.124059895681956), ([748, 1724], 25.169889867924677), ([51, 2407], 25.204861104576345

In [18]:
@load "../../Data/tmp/wip_test.jld2" input output
output["IFILs"]

96-element Vector{Any}:
 [([748, 1725], 23.92455349903287), ([748, 1724], 23.96354401541596), ([748, 1718], 24.28196954629767), ([846, 1724], 24.752825160736354), ([846, 1725], 24.800319910112158), ([740, 1725], 25.10700485325493), ([737, 1718], 25.213469938876298), ([869, 1724], 25.32574800184744), ([703, 1718], 25.45647706768041), ([740, 1718], 25.462829822196742)  …  ([798, 1464, 2004, 2017], 46.36206014270773), ([798, 1509, 2003, 2004], 46.4520948255289), ([810, 2038, 2067, 2098], 47.30434541547556), ([469, 789, 1566, 1995], 47.457641911347004), ([468, 1566, 1675, 1995], 47.62720496168401), ([810, 831, 2038, 2098], 48.301855104675305), ([468, 1205, 1674, 1995], 49.14033289963158), ([797, 798, 2003, 2004], 50.84849316210928), ([469, 789, 1995, 2003], 54.48503911056762), ([468, 469, 1566, 1995], 54.77862226711244)]
 [([51, 1718], 25.014294323285053), ([50, 1718], 25.098467849699105), ([748, 1725], 25.124059895681956), ([748, 1724], 25.169889867924677), ([51, 2407], 25.204861104576345

In [9]:
println("Initial energy: $(output["Es"][1]) | final energy: $(output["Es"][end]) | minimal energy: $(minimum(output["Es"]))")

Initial energy: 8115.9460200299145 | final energy: 8055.157870223171 | minimal energy: 8050.509902123483


In [ ]:
try
    rm("assets/output", recursive = true)
catch 
end
mkpath("assets/output")
for (i, state) in enumerate(output["states"])
    MorphoMolNotebooks.configuration_to_poly(MorphoMol.Utilities.get_flat_realization(state, template_centers), radii, "assets/output/$(i)", n_mol, n_atoms_per_mol)
end

In [11]:
function generate_ouput_plots(sim_out)
    paired_colors = cgrad(:Paired_12, 12, categorical = true)
    xs = 1:length(sim_out["Vs"])

    pt_per_in = 72
    width =  5*5
    height = 5*3
    f_size = (pt_per_in * width, pt_per_in * height)
    fs = 15
    border_width = 0.5
    fig = Figure(size = f_size, fontsize = fs)
    ms = 4
    scatter(fig[1, 1], xs, sim_out["Vs"], markersize = ms, color = paired_colors[1], axis = (ylabel = L"V", ylabelrotation = 0,))
    scatter(fig[1, 2], xs, sim_out["As"], markersize = ms, color = paired_colors[3], axis = (ylabel = L"A", ylabelrotation = 0,))
    scatter(fig[1, 3], xs, sim_out["Cs"], markersize = ms, color = paired_colors[5], axis = (ylabel = L"C", ylabelrotation = 0,))
    scatter(fig[1, 4], xs, sim_out["Xs"], markersize = ms, color = paired_colors[7], axis = (ylabel = L"X", ylabelrotation = 0,))
    scatter(fig[2, 1], xs, sim_out["OLs"], markersize = ms, color = paired_colors[10], axis = (ylabel = L"OL", ylabelrotation = 0,))
    scatter(fig[2, 2], xs, sim_out["Es"], markersize = ms, color = paired_colors[12], axis = (ylabel = L"F_{sol}", ylabelrotation = 0,))
    #save(string("$(save_dir)/hmc_output_plots.png"), fig, pt_per_unit = 1)
    fig
end

save_output_plots (generic function with 1 method)

In [12]:
generate_ouput_plots(output)

CairoMakie.Screen{IMAGE}
